# Here we just steal tests from Sam

## 0. Imports and misc

In [ ]:
import itertools as it; import more_itertools as mit
import functools as ft
import os
import sys
from pathlib import Path

import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
from numpy.random import default_rng; rng=default_rng()
import seaborn as sns
from sklearn.utils._testing import ignore_warnings #green flag: good code ahead
from sklearn.exceptions import ConvergenceWarning
import torch
from torch import nn, Tensor
import torch.nn.functional as F
from torch.utils.tensorboard import SummaryWriter
from tqdm.notebook import trange, tqdm
import torch.utils.data

from ot_markov_distances import discounted_wl_infty
from ot_markov_distances.utils import weighted_transition_matrix
from ml_lib.misc import auto_repr, all_equal


def append_path(s):
    s = os.path.abspath(s)
    if s in sys.path: return
    sys.path.insert(0,s)
append_path("")

from sklearn.ensemble import AdaBoostClassifier
from sklearn.svm import SVC
from sklearn.manifold import TSNE

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, StratifiedKFold
import numpy as np
#from svm import grakel_to_nx, grakel_to_igraph
from grakel import GraphKernel, Graph
import itertools
#from grakel.datasets import fetch_dataset
import grakel.datasets
import networkx as nx
import time
import ot
import sys
import wwl
import igraph as ig
from tqdm import trange, tqdm
import cProfile
import re
import multiprocessing as mp
import matplotlib.pyplot as plt
%load_ext autoreload
%autoreload 2

device = torch.device("cuda:2")

OK just monkey-patch not working function

In [4]:
from unittest.mock import patch
from urllib.request import urlopen, HTTPError
import ssl
from shutil import copyfileobj

#monkey patch this
def _download_zip(url, output_name):
    """Download a file from a requested url and store locally.
    Parameters
    ----------
    url : str
        The url from where the file will be downloaded.
    output_name : str
        The name of the file in the local directory.
    Returns
    -------
    None.
    """
    #ctx = ssl.create_default_context(ssl.Purpose.CLIENT_AUTH)
    context = ssl._create_unverified_context()
    filename = output_name + ".zip"
    try:
        data_url = urlopen(url, context=context)
    except HTTPError as e:
        if e.code == 404:
            e.msg = "Dataset '%s' not found on mldata.org." % output_name
        raise

    # Store Zip File
    try:
        with open(filename, 'w+b') as zip_file:
            copyfileobj(data_url, zip_file)
    except Exception:
        os.remove(filename)
        raise
    data_url.close()
    


And steal some stuff from Sam

In [8]:
def grakel_to_nx(G, include_attr=True):
    nx_G = []
    for graph in G:
        adj_mat = graph.get_adjacency_matrix()
        nx_graph = nx.from_numpy_array(adj_mat)
        nodes = sorted(list(graph.node_labels.keys()))
        if include_attr == True:
            for i in range(adj_mat.shape[0]):
                nx_graph.nodes[i]["attr"] = graph.node_labels[nodes[i]]
        nx_G.append(nx_graph)
    return nx_G

def grakel_to_igraph(G, add_attr=False):
    lst = []
    attr_list = []
    max_nodes = 0
    szs = []
    for graph in G:
        adj_mat = graph.get_adjacency_matrix()
        igraph = ig.Graph.Adjacency(adj_mat)
        n = adj_mat.shape[0]
        if add_attr:
            nodes = sorted(list(graph.node_labels.keys()))
            attrs = []
            for i in range(len(nodes)):
                attrs.append(graph.node_labels[nodes[i]])
            igraph.vs["label"] = attrs
        if n > max_nodes:
            max_nodes = n
        lst.append(igraph)
    if add_attr:
        for graph in G:
            nodes = sorted(list(graph.node_labels.keys()))
            attrs = [0]*max_nodes
            for i in range(graph.n):
                attrs[i] = graph.node_labels[nodes[i]]
            attr_list.append(attrs)
    return lst, attr_list

## 1. fetch data

In [66]:
dataset_titles = ["MUTAG", "PTC_MR", "PTC_FM",  "COX2", "PROTEINS", "PROTEINS_full", "ENZYMES"]
ds_names = [ "mutag", "ptc_mr", "ptc_fm","cox2", "proteins", "proteins_full", "ENZYMES"]

In [67]:
datasets = []
for dataset_name, dataset_title in zip(ds_names, dataset_titles):
    with patch("grakel.datasets.base._download_zip", _download_zip):    
        ds = grakel.datasets.fetch_dataset(dataset_title, as_graphs=True)
    G = ds.data
    nx_G = grakel_to_nx(G, include_attr=True)
    #igraphs, graph_attrs = grakel_to_igraph(G, add_attr=False)
    y = ds.target
    datasets.append((nx_G, y))

Extracting dataset  MUTAG..
Parsing dataset  MUTAG..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Extracting dataset  PTC_MR..
Parsing dataset  PTC_MR..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Extracting dataset  PTC_FM..
Parsing dataset  PTC_FM..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Extracting dataset  COX2..
Parsing dataset  COX2..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Extracting dataset  PROTEINS..
Parsing dataset  PROTEINS..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Extracting dataset  PROTEINS_full..
Parsing dataset  PROTEINS_full..
Parse was succesful..
Deleting unzipped dataset files..
Going back to the original directory..
Extracting dataset  ENZYMES..
Parsing dataset  ENZYMES..
Parse was succesful..
Deleting 

In [30]:
type(ds.target)

numpy.ndarray

In [32]:
dataset_index = -1
data, gt = datasets[dataset_index]
data[0].nodes[2]

{'attr': 0}

## 2. Functions 

In [73]:
def compute_distance_matrix( set1, set2, wl_parameters, device=device, q=.1):
    distance_matrix = torch.zeros(len(set1), len(set2), device=device)
    
    if set1 is set2:
        #only compute the upper triangular part
        iterator = it.combinations(enumerate(set1), 2)
        iterator = (((clip1_i, clip1), (clip2_i, clip2))
                    for (clip1_i, clip1), (clip2_i, clip2) in iterator
                    if clip1_i != clip2_i
                    )
        n_iter = (len(set1) * (len(set1) - 1) // 2 )
    else: 
        iterator = it.product(enumerate(set1), enumerate(set2))
        n_iter = len(set1) * len(set2)
    iterator = tqdm(iterator, total = n_iter)
    
    for (clip1_i, clip1), (clip2_i, clip2) in iterator:
        #distance matrix
        attr1 = torch.as_tensor(list(nx.get_node_attributes(clip1, "attr").values()), device=device)
        attr2 = torch.as_tensor(list(nx.get_node_attributes(clip2, "attr").values()), device=device)
        if torch.is_floating_point(attr1):
            D = (attr1[:, None] - attr2[None, :]).square()
            if len(D.shape)>2:
                D = D.mean(*range(2, D.shape))
        else:
            D = (attr1[:, None] == attr2[None, :])
        D = D.to(torch.float32)

        #markov chains
        markov1 = weighted_transition_matrix(clip1, q=q)
        markov2 = weighted_transition_matrix(clip2, q=q)
        
        #distributions
        mu1 = torch.ones(1, len(clip1), device=device) / len(clip1)
        mu2 = torch.ones(1, len(clip2), device=device) / len(clip2)
        
        #distance
        distance_matrix[clip1_i, clip2_i] = \
            discounted_wl_infty(markov1.to(device)[None, ...], 
                                markov2.to(device)[None, ...], 
                                D[None, ...], 
                                muX=mu1,
                                muY=mu2,
                                **wl_parameters).square()
        
    if set1 is set2:
        distance_matrix = distance_matrix + distance_matrix.T
    return distance_matrix


100%|██████████| 17578/17578 [57:47<00:00,  5.07it/s] 


tensor([[0.0000, 0.2851, 0.2843,  ..., 0.2583, 0.4739, 0.3491],
        [0.2851, 0.0000, 0.1703,  ..., 0.1506, 0.3204, 0.2203],
        [0.2843, 0.1703, 0.0000,  ..., 0.1498, 0.3196, 0.2196],
        ...,
        [0.2583, 0.1506, 0.1498,  ..., 0.0000, 0.2919, 0.1969],
        [0.4739, 0.3204, 0.3196,  ..., 0.2919, 0.0000, 0.3884],
        [0.3491, 0.2203, 0.2196,  ..., 0.1969, 0.3884, 0.0000]],
       device='cuda:7')

In [87]:
# function stolen from Sam
@ignore_warnings(category=ConvergenceWarning)
def choose_parameters( gammas, Cs, D, y, cv=5, ksvm=False):
    cv = StratifiedKFold(n_splits=cv)
    results = []
    param_pairs = []
    for g in gammas:
        for c in Cs:
            param_pairs.append((g, c))

    for train_index, test_index in cv.split(D, y):
        split_results = []
        for i in range(len(gammas)):
            for j in range(len(Cs)):
                g = gammas[i]
                c = Cs[j]
                D_train = D[train_index][:, train_index]
                D_test = D[test_index][:, train_index]
                K_train = np.exp(-g * D_train)
                K_test = np.exp(-g * D_test)
                y_train = y[train_index]
                y_test = y[test_index]
                clf = SVC(kernel='precomputed', C = c, max_iter=1000)
                clf.fit(K_train, y_train)
                y_pred = clf.predict(K_test)
                split_results.append(accuracy_score(y_test, y_pred))
        results.append(split_results)

    results = np.array(results)
    fin_results = results.mean(axis=0)
    best_idx = np.argmax(fin_results)
    return param_pairs[best_idx]

## 3. Execution

In [97]:
gammas = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
Cs = [0.001, 0.01, 0.1, 1, 10, 100, 1000]
skf = StratifiedKFold(n_splits=10, shuffle=True)


In [109]:
wl_parameters=dict(delta=.2, 
                   x_is_sparse=True, 
                   y_is_sparse=True,
                   convergence_threshold_rtol = .01,
                   convergence_threshold_atol = 1e-3)
def run_experiment(data, y, distances):
    accuracies = []
    for train_index, test_index in tqdm(skf.split(distances, y)):
        D_train = distances[train_index][:, train_index]
        D_test = distances[test_index][:, train_index]
        y_train = y[train_index]
        y_test = y[test_index]
        params = choose_parameters(gammas, Cs, D_train, y_train, cv=10)
        clf = SVC(kernel='precomputed', C = params[1], max_iter=5000)
        K_train = np.exp(-params[0]*D_train)
        K_test = np.exp(-params[0]*D_test)
        clf.fit(K_train, y_train)
        y_pred = clf.predict(K_test)
        accuracies.append(accuracy_score(y_test, y_pred))
    return accuracies

In [ ]:
results = []
for (data, y), name, i in zip(datasets, dataset_titles, it.count()):
    if distance_matrices[i] is None:
        distance_matrices[i] = compute_distance_matrix(data, data, wl_parameters).numpy(force=True)
    distance_matrix = distance_matrices[i]
    accuracies = run_experiment(data, y, distance_matrix)
    #print("Done with WWL experiments")
    print(f"{name}: Average accuracy:", np.mean(accuracies), "Standard deviation:", np.std(accuracies))
    results.append(accuracies)


10it [00:14,  1.41s/it]


MUTAG: Average accuracy: 0.6807017543859649 Standard deviation: 0.05309735421200547


  0%|          | 0/58996 [00:00<?, ?it/s]/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/ot_markov_distances/utils.py:27: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A: Tensor= torch.as_tensor(nx.adjacency_matrix(G, weight=None).todense())#type:ignore
 97%|█████████▋| 57380/58996 [5:26:32<09:56,  2.71it/s]  IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

  8%|▊         | 4915/60726 [24:43<6:18:27,  2.46it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.i

PTC_FM: Average accuracy: 0.6128571428571429 Standard deviation: 0.05285714285714286


  0%|          | 0/108811 [00:00<?, ?it/s]/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/ot_markov_distances/utils.py:27: FutureWarning: adjacency_matrix will return a scipy.sparse array instead of a matrix in Networkx 3.0.
  A: Tensor= torch.as_tensor(nx.adjacency_matrix(G, weight=None).todense())#type:ignore
  1%|          | 697/108811 [04:46<12:36:58,  2.38it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

  3%|▎         | 3457/108811 [23:38<11:42:36,  2.50it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.

In [111]:
results

[autoreload of ot_markov_distances.sinkhorn failed: Traceback (most recent call last):
  File "/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 261, in check
    superreload(m, reload, self.old_objects)
  File "/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 484, in superreload
    update_generic(old_obj, new_obj)
  File "/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 333, in update_class
    if update_generic(old_obj, new_obj):
  File "/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/IPython/extensions/autoreload.py", line 381, in update_generic
    update(a, b)
  File "/home/tristan/miniconda3/envs/default/lib/python3.10/site-packages/IPyth

[[0.631578947368421,
  0.7894736842105263,
  0.631578947368421,
  0.6842105263157895,
  0.7368421052631579,
  0.631578947368421,
  0.7368421052631579,
  0.631578947368421,
  0.6666666666666666,
  0.6666666666666666],
 [0.5428571428571428,
  0.5428571428571428,
  0.5714285714285714,
  0.5714285714285714,
  0.5588235294117647,
  0.5588235294117647,
  0.5588235294117647,
  0.5588235294117647,
  0.5588235294117647,
  0.5588235294117647],
 [0.6571428571428571,
  0.6,
  0.5714285714285714,
  0.6,
  0.6,
  0.6285714285714286,
  0.6,
  0.6857142857142857,
  0.6857142857142857,
  0.5],
 [0.7872340425531915,
  0.7872340425531915,
  0.7872340425531915,
  0.7872340425531915,
  0.7872340425531915,
  0.7659574468085106,
  0.7659574468085106,
  0.782608695652174,
  0.782608695652174,
  0.782608695652174],
 [0.6160714285714286,
  0.7678571428571429,
  0.6696428571428571,
  0.6216216216216216,
  0.5855855855855856,
  0.6576576576576577,
  0.7027027027027027,
  0.6666666666666666,
  0.6036036036036037,


In [34]:
WL = grakel.WeisfeilerLehman()

In [36]:
WL.fit_transform(ds)

ValueError: Unsupported input type. For more information check the documentation, concerning valid input types for graph type object.

In [45]:
sys.path.append(f"{os.environ['HOME']}/software/tudataset")

In [49]:
from tud_benchmark import kernel_baselines


In [61]:
test_ds = "ENZYMES"
classes = dp.get_dataset(test_ds)
gram_matrix = kernel_baselines.compute_wl_1_dense(test_ds, 3, True, False)


In [62]:
gram_matrix

array([[ 964.,  502.,  585., ...,  973.,  998.,  983.],
       [ 502.,  408.,  362., ...,  591.,  615.,  610.],
       [ 585.,  362.,  546., ...,  587.,  678.,  674.],
       ...,
       [ 973.,  591.,  587., ..., 2054., 1417., 1387.],
       [ 998.,  615.,  678., ..., 1417., 1596., 1501.],
       [ 983.,  610.,  674., ..., 1387., 1501., 1652.]])

array([6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6,
       6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 6, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5,
       5, 5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,

In [53]:
cd /home/tristan/software/tudataset/tud_benchmark

/home/tristan/software/tudataset/tud_benchmark


In [ ]:
get_